In [14]:
import plac
from tqdm import tqdm
import numpy as np
from PIL import Image
import torch
from torch import nn
import torch.nn.functional as F
import matplotlib.pyplot as plt
import pdb; 
import math
from math import nan

In [128]:
class Generator_d(nn.Module):

  def __init__(self, net_param):

    super(Generator_d, self).__init__()
    self.f = []
    for i in range(1,len(net_param)):
      net_param[i-1]
      layer = nn.Linear(net_param[i-1], net_param[i], bias = False)
      nn.init.normal_(list(layer.parameters())[0].data, 0, 1/np.sqrt(net_param[i]))
      self.f = np.append(self.f, [layer])
    
  def decoder(self, z):
    for i in range(len(self.f)):
      z = F.relu(self.f[i](z))
    return z 
  
  def forward(self, z):
    return self.decoder(z)


In [113]:
expanse_param = 5
z_dim = 20
depth = 5
net_param = np.append([z_dim], np.ones(depth)*z_dim*expanse_param)
net_param.astype(int)
t = np.ones(10)
k = [1, 1, 1]
len(t)

10

In [ ]:
class Generator3(nn.Module):

  def __init__(self, net_param):

    super(Generator3, self).__init__()
    self.f1 = nn.Linear(net_param[0], net_param[1], bias = False)
    self.f2 = nn.Linear(net_param[1], net_param[2], bias = False)
    self.f3 = nn.Linear(net_param[2], net_param[3], bias = False)
    
  def decoder(self, z):
    z = F.relu(self.f1(z))
    z = F.relu(self.f2(z))
    z = F.relu(self.f3(z))
    return z 

  # def decoder(self, z):
  #   z = self.f1(z)
  #   z = self.f2(z)
  #   z = self.f3(z)
  #   return z  
  
  def forward(self, z):
    return self.decoder(z)

def weights_init(m):
  classname = m.__class__.__name__

  # for every Linear layer in a model..
  if classname.find('Linear') != -1: 
    n = m.out_features     # get the number of the inputs
    y = 1.0/np.sqrt(n)
    m.weight.data.normal_(0, y)

class Generator3_sigmoid(nn.Module):

  def __init__(self, net_param):

    super(Generator3_sigmoid, self).__init__()
    self.f1 = nn.Linear(net_param[0], net_param[1], bias = False)
    self.f2 = nn.Linear(net_param[1], net_param[2], bias = False)
    self.f3 = nn.Linear(net_param[2], net_param[3], bias = False)
    
  def decoder(self, z):
    z = F.sigmoid(self.f1(z))
    z = F.sigmoid(self.f2(z))
    z = F.sigmoid(self.f3(z))
    return z 
  
  def forward(self, z):
    return self.decoder(z)


In [134]:
class algo_parameter:
  def __init__(self, stepsize, max_itt, tolerance):
    self.stepsize = stepsize
    self.max_itt = max_itt
    self.tolerance = tolerance
    
class weight_mats():
  W = {} # create a data struct for weight matrices
  def __init__(self, net_param):
    for i in range(len(net_param)-1):
      self.W[i] =torch.randn(net_param[i+1],net_param[i])

In [ ]:
## function that solves gradient descent algorithm withough flipping with stoping criteria as successive error
def gradient_desc(net_param, algo_param, generator, y0, A, out_toggle):
  x_dim = net_param[len(net_param)-1]
  z_dim = net_param[0]
  device = "cuda"
  z = torch.randn(z_dim, device=device, requires_grad = True)
  z_temp = torch.randn(z_dim, device=device, requires_grad = False)
  succ_error = 1
  itt = 1
  while itt < algo_param.max_itt and succ_error > algo_param.tolerance:

    z_temp.data = z.clone()
    y_pred = torch.matmul(A, generator(z).reshape(x_dim))
    loss_fn = (y_pred-y0).pow(2).mean()
    
    loss_fn.backward()
    grad_norm = torch.norm(z.grad)
    with torch.no_grad():
      z -= algo_param.stepsize*z.grad
      z.grad.zero_()

    itt += 1
    succ_error = torch.norm(z_temp-z)/torch.norm(z)  
    
    # pdb.set_trace()

    if out_toggle != 0:
      if itt % ((out_toggle // 10)*10 +1) == 0:
        print('====> In gradient descent: Iteration: {} Successive error: {:.4e}'.format(itt, succ_error))

  return z, itt

## function that solves gradient descent algorithm withough flipping with stopping criteria as relative error
def gradient_desc_rel(net_param, algo_param, generator, y0, A, z0, out_toggle):
  x_dim = net_param[len(net_param)-1]
  z_dim = net_param[0]
  device = "cuda"
  z = torch.randn(z_dim, device=device, requires_grad = True)
  rel_error = 1
  itt = 1
  while itt < algo_param.max_itt and rel_error > algo_param.tolerance:

    y_pred = torch.matmul(A, generator(z).reshape(x_dim))
    loss_fn = (y_pred-y0).pow(2).mean()
    
    loss_fn.backward()
    grad_norm = torch.norm(z.grad)
    with torch.no_grad():
      z -= algo_param.stepsize*z.grad
      z.grad.zero_()

    itt += 1
    rel_error = torch.norm(z0-z)/torch.norm(z0)  
    
    # pdb.set_trace()

    if out_toggle != 0:
      if itt % ((out_toggle // 10)*10 +1) == 0:
        print('====> In gradient descent: Iteration: {} Successive error: {:.4e}'.format(itt, rel_error))

  return z, itt

In [ ]:
## function that solves quasi-gradient algorithm with successive error as stopping criteria
def quasi_grad(net_param, algo_param, generator, y0, weight_mats_d3, A, out_toggle):
  x_dim = net_param[len(net_param)-1]
  z_dim = net_param[0]
  [m,_] = A.size()
  Wm = weight_mats_d3.W[0]
  scale = 1 # this will change if the weight matrices are not normalized
  depth = len(weight_mats_d3.W)
  for i in range(len(weight_mats_d3.W) - 1):
    Wm = torch.matmul(weight_mats_d3.W[i+1],Wm)
  zk = torch.randn(1, z_dim).cuda()
  # zk = .5*zk/torch.norm(zk,2)
  z_temp = zk
  
  itt = 0
  succ_error = 1
  # pdb.set_trace()
  while itt < algo_param.max_itt and succ_error > algo_param.tolerance:
    xk = generator(zk).reshape(x_dim)
    d = torch.matmul(torch.t(torch.matmul(A,Wm)), torch.matmul(A,xk) -  y0.reshape(m)) * (scale**(-1))
    step = algo_param.stepsize *  d
    z_temp = zk - torch.t(step)
    itt += 1

    succ_error = torch.norm(z_temp-zk)/torch.norm(zk)
    zk = z_temp
    
    if out_toggle != 0:
      if itt % ((out_toggle // 10)*10 +1) == 0:
        print('====> In quasi-gradient: Iteration: {} Successive error: {:.4e}'.format(itt, succ_error))
  return zk, itt

## function that solves quasi-gradient algorithm with relative error as stopping criteria
def quasi_grad_rel(net_param, algo_param, generator, y0, weight_mats_d3, A, z0, out_toggle):
  x_dim = net_param[len(net_param)-1]
  z_dim = net_param[0]
  [m,_] = A.size()

  Wm = weight_mats_d3.W[0]
  scale = 1 # this will change if the weight matrices are not normalized
  depth = len(weight_mats_d3.W)
  for i in range(len(weight_mats_d3.W) - 1):
    Wm = torch.matmul(weight_mats_d3.W[i+1],Wm)
  # zk = torch.zeros(1, z_dim).cuda()
  zk = torch.zeros(1, z_dim)
  
  itt = 0
  rel_error = 1
  while itt < algo_param.max_itt and rel_error > algo_param.tolerance:
    xk = generator(zk).reshape(x_dim)
    d = torch.matmul(torch.t(torch.matmul(A,Wm)), torch.matmul(A,xk) -  y0.reshape(m)) * (scale**(-1))
    step = algo_param.stepsize *  d
    zk = zk - torch.t(step)
    itt += 1
    rel_error = torch.norm(z0-zk)/torch.norm(z0)

    
    if out_toggle != 0:
      if itt % ((out_toggle // 10)*10 +1) == 0:
        print('====> In quasi-gradient: Iteration: {} Relative error: {:.4e}'.format(itt, rel_error))
  if out_toggle != 0:   
    print('====> In quasi-gradient: Iteration: {} Relative error: {:.4e}'.format(itt, rel_error))  
    print('')
  # pdb.set_trace()
  return zk, itt